In [1]:
import pandas as pd
import numpy as np
import re
import copy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)  # Displays all the text for each value in a column
pd.set_option('display.float_format', '{:,.2f}'.format)

def format_bigquery_column_names(nftfi):
    # Remove special charachters
    nftfi.columns = [re.sub(r'[^a-zA-Z0-9\s]+', '', column) for column in nftfi.columns]
    # Remove spaces at beginning and end
    nftfi.columns = nftfi.columns.str.strip()
    # Replace spaces with underscore
    nftfi.columns = nftfi.columns.str.replace(' ', '_')
    # Define a function to convert camel-case to kebab-case
    def camel_to_kebab(s):
        # Replace consecutive capital letters with a single lowercase letter
        s = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', s)
        # Convert remaining camel-case string to kebab-case
        s = re.sub(r'(?<!^)(?<!_)(?=[A-Z])(?![A-Z])', '_', s).lower()
        return s
    # Apply the function to all column names
    nftfi.columns = nftfi.columns.map(camel_to_kebab)
    return nftfi



## Load tables

In [2]:
# try: 
#     mq = pd.read_csv('MQ_from_metaquants_telegram.csv')
# except FileNotFoundError:
#     mq = pd.read_csv('analysis/metaquants_analysis/MQ_from_metaquants_telegram.csv')

In [3]:
try: 
    mq = pd.read_csv('data/metaquants_loans.csv')
except FileNotFoundError:
    mq = pd.read_csv('../../data/metaquants_loans.csv')

In [4]:
mq = mq.sort_values(by='block_timestamp', ascending=False)
display(mq.head())
display(mq.protocol.unique())
mq = mq.loc[mq['protocol'] == 'nftfi']
display(mq.protocol.unique())

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p
12757,0x0bf87b1d9ecec93f80c864f41899cb206c261c30adad5999318fd42c7ef294f8,2023-05-18 10:16:23.000000 UTC,36207,0xb04a68660362b3c10a6745b14d383a27503c3d50,0x9cd074f0192bb2a477da3271ba03db4fc5cd38c7,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 10:16:23.000000 UTC,30.00,0.50,8231,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,nftfi,"1,095.78",False,17285778,p2p
16031,0xf660371bc68bfdcf8810846fa8b418147f4091081fba11c0cfdefa5ddcef8fd4,2023-05-18 09:23:11.000000 UTC,36204,0xc78c3f6cec7d593e0fd931670ca820aac4eee08c,0x5b20dce335d131c6de0cc11b481ace2aade788df,1.30,1.37,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 09:23:11.000000 UTC,30.00,0.70,3000118,0x64780ce53f6e966e18a22af13a2f97369580ec11,nftfi,"2,374.36",False,17285519,p2p
17508,0x1bc701828c5f3b8e1e753ae095cd0487e9f431545c816fbe27110a639de53ba7,2023-05-18 08:51:47.000000 UTC,36201,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 08:51:47.000000 UTC,9.00,0.36,12015,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,740.40",False,17285365,p2p


array(['nftfi'], dtype=object)

array(['nftfi'], dtype=object)

In [5]:
# try: 
#     nftfi = pd.read_csv('../../analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# except FileNotFoundError:
#     nftfi = pd.read_csv('analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# nftfi = format_bigquery_column_names(nftfi)
# nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})
# 
# # Drop nftfi gas price related columns
# for col in nftfi.columns:
#     if 'gas' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)
#     if 'fee' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)

In [6]:
try: 
    nftfi = pd.read_csv('loans_raw_loan_input_datanftfi_loans.csv')
except FileNotFoundError:
    nftfi = pd.read_csv('loans_raw_loan_input_data.csv')
raw_nftfi = copy.deepcopy(nftfi)
nftfi = format_bigquery_column_names(nftfi)
nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})

# Drop nftfi gas price related columns
for col in nftfi.columns:
    if 'gas' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)
    if 'fee' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)

/tmp/ipykernel_709936/3619570390.py:4: DtypeWarning: Columns (7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  nftfi = pd.read_csv('loans_raw_loan_input_data.csv')


### Subselect each dataset to match their start and end dates based on who has most data

In [7]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

if nftfi['date'].min() > mq['block_timestamp'].min():
    # MQ has more complete dataset
    mq = mq.loc[mq['block_timestamp'] >= nftfi['date'].min()]
else:
    # NFTfi has more complete dataset
    nftfi = nftfi.loc[nftfi['date'] >= mq['block_timestamp'].min()]

if nftfi['date'].max() > mq['block_timestamp'].max():
    # NFTfi has more complete dataset
    nftfi = nftfi.loc[nftfi['date'] <= mq['block_timestamp'].max()]
else:
    # MQ has more complete dataset
    mq = mq.loc[mq['block_timestamp'] <= nftfi['date'].max()]

print('\n\n --------------- AFTER')
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2020-10-15 18:15:24.000000 UTC; max mq date: 2023-05-18 12:41:47.000000 UTC
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-30 07:02:11+00:00


 --------------- AFTER
min mq date: 2022-03-30 10:21:32.000000 UTC; max mq date: 2023-05-18 12:41:47.000000 UTC
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-18 12:41:47+00:00


In [8]:
print('MQ')
display(mq.head(2))
print('\n--------------------------------------------------------------------')
print('\n\nNFTfi')
display(nftfi.head(2))

MQ


,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p



--------------------------------------------------------------------


NFTfi


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN


In [9]:
mq['loan_id'].head(2)

17507    36220
9936     36212
Name: loan_id, dtype: int64

### Clean data

In [10]:
# clean loan IDs to try and compare them
mq['loan_id_cleaned'] = mq['loan_id'].astype(str).str.replace('-','')
nftfi['loan_no_cleaned'] = nftfi['loan_no'].str.split('-').apply(lambda x: x[-1])
nftfi

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z,1
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN,2
2,2022-03-30 10:43:05+00:00,v2.loan.fixed-3,2022-03-30T10:43:05.000Z,2022-03-30T10:50:17.000Z,True,0.01,True,10000000000000000,10000100000000000,NaN,NaN,"100,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,73.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,2022-03-30T10:59:51.000Z,3
3,2022-03-30 12:58:30+00:00,v2.loan.fixed-4,2022-03-30T12:58:30.000Z,2022-04-13T12:58:30.000Z,True,14.00,True,100000000000000,101000000000000,NaN,NaN,"1,000,000,000,000.00",NaN,0x47842f8b1263D880c43743FBD33D9E8dcde43a91,0x712AAc15cE295C9e503cf0D579FEF4a910644Ccf,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,184000128,False,26.07,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T13:01:53.000Z,2022-01-25T10:39:14.000Z,4
4,2022-03-30 14:19:41+00:00,v2.loan.fixed-5,2022-03-30T14:19:41.000Z,2022-04-13T14:19:41.000Z,True,14.00,False,30000000000000000000,30230137000000000000,NaN,NaN,"230,137,000,000,000,000.00",NaN,0x47842f8b1263D880c43743FBD33D9E8dcde43a91,0x712AAc15cE295C9e503cf0D579FEF4a910644Ccf,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,184000128,False,20.00,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-03-30T14:28:29.000Z,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36216,2023-05-18 12:37:35+00:00,v2.loan.fixed.collection-36216,2023-05-18T12:37:35.000Z,2023-06-17T12:37:35.000Z,False,30.00,False,"500,000,000,000,000,000.00","528,356,000,000,000,000.00",NaN,NaN,28356000000000000,NaN,0xc3cea12ffda8B3Dff435155461de6FCc72315117,0xcF82579cAB223Cc1e467C17e371f8027C1767667,0xB852c6b5892256C264Cc2C888eA462189154D8d7,3258,True,69.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,36216
36217,2023-05-18 12:37:59+00:00,v2.loan.fixed.collection-36217,2023-05-18T12:37:59.000Z,2023-06-17T12:37:59.000Z,False,30.00,False,"500,000,000,000,000,000.00","528,356,000,000,000,000.00",NaN,NaN,28356000000000000,NaN,0xc3cea12ffda8B3Dff435155461de6FCc72315117,0x040Dedbf67cAcdfa8B6FfBAeC08AD1B1D83F633B,0xB852c6b5892256C264Cc2C888eA462189154D8d7,8080,True,69.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,36217
36218,2023-05-18 12:38:23+00:00,v2.loan.fixed.collection-36218,2023-05-18T12:38:23.000Z,2023-06-17T12:38:23.000Z,False,30.00,False,"500,000,000,000,000,000.00","528,356,000,000,000,000.00",NaN,NaN,28356000000000000,NaN,0xc3cea12ffda8B3Dff435155461de6FCc72315117,0xcF82579cAB223Cc1e467C17e371f8027C1767667,0xB852c6b5892256C264Cc2C888eA462189154D8d7,4398,True,69.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,36218
36219,2023-05-18 12:39:23+00:00,v2-1.loan.fixed-36219,2023-05-18T12:39:23.000Z,2023-06-17T12:39:23.000Z,False,30.00,False,"2,331,300,000,000,000,000.00","2,348,545,232,876,712,448.00",NaN,NaN,17245232876712448,NaN,0x47a0DFeb07AbeBd5f77aA5ffAA18fAeCd7686b4F,0x

In [11]:
display(mq['loan_id_cleaned'])
display(nftfi['loan_no_cleaned'])

17507    36220
9936     36212
12757    36207
16031    36204
17508    36201
         ...  
12520     6191
584          3
745          2
7527      6190
585          1
Name: loan_id_cleaned, Length: 19547, dtype: object

0            1
1            2
2            3
3            4
4            5
         ...  
36216    36216
36217    36217
36218    36218
36219    36219
36220    36220
Name: loan_no_cleaned, Length: 36221, dtype: object

In [12]:
mq.shape[0], nftfi.shape[0]

(19547, 36221)

In [13]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2022-03-30 10:21:32.000000 UTC; max mq date: 2023-05-18 12:41:47.000000 UTC
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-18 12:41:47+00:00


In [14]:
mq.loc[mq.loan_id_cleaned.duplicated()==True].sort_values('loan_id_cleaned')

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
7104,0xbbd45783e2cd294f86ec604831eeeca7be8bd844818ef7ea52a3df7eb5a01c94,2023-01-18 06:03:35.000000 UTC,21523,0x45d1a3acee4b57f5bb078a14d993364f80e86657,0xebd0db1bfaa50e7abe52412644e968166f349755,0.90,0.92,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-02-01 06:03:35.000000 UTC,14.00,0.69,1662,0x394e3d3044fc89fcdd966d3cb35ac0b32b0cda91,nftfi,"1,428.82",False,16431774,p2p,21523
21675,0xfb83f7b98831eec2595e1af36e064057cfac3018b15513a6cb8dd7017c6c4df8,2023-02-06 20:30:11.000000 UTC,24224,0x910d44c699c24cddd03ce8e7e8cc40835464fb89,0x35fa3c19cf39674353025234ed001bcbb36af35c,0.31,0.32,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-08 20:30:11.000000 UTC,30.00,0.55,32395,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,511.04,False,16572143,p2p,24224
16114,0xd5f969b32332020a8e7129e4aa2f20a5bff446b31e0054469aa47013268704d7,2023-03-26 16:43:11.000000 UTC,30521,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:11.000000 UTC,14.00,0.79,323,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912862,p2p,30521
16111,0x205c9f50b22b5a4751b14e09efb1eacc757ede76ca421a6b500a0962155ed4e4,2023-03-26 16:43:47.000000 UTC,30522,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:47.000000 UTC,14.00,0.79,365,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912865,p2p,30522
21042,0x5d215283e0326ebddc63178230b83a1b00612e9fe30de12c4e52aaea4b2c1879,2023-03-26 16:44:11.000000 UTC,30523,0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06,0x1797b4235473fbe0e7e44322f01c1b5618ebda41,1.00,1.02,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-25 16:44:11.000000 UTC,30.00,0.29,14093,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,"1,764.35",False,16912867,p2p,30523
7527,0xbd19dbb314cdcc2be0c0c579e639596635e77b7d5e78062e3d14ae27afa12266,2022-03-30 10:24:07.000000 UTC,-6190,0x52ac5997d2ac830d54b4bad431ab95546d279144,0x25f9ee5543d3d815051fcbedffd283ce457f7426,"1,500.00","1,570.00",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-04-29 10:24:07.000000 UTC,30.00,0.57,462,0x3fe1a4c1481c8351e91b64d5c398b159de07cbc5,nftfi,"1,500.00",False,14487034,p2p,6190
12520,0x504e78de6e5a1aa3b3c54f4a681a6f7866ba9c827ff04cabd4c7d986bde13dc7,2022-03-30 11:03:13.000000 UTC,-6191,0x8854631995b99bbfca16b8ea339538950181132a,0x85db67812169378666547e03b293785d7d9a6b06,3.20,3.31,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:03:13.000000 UTC,30.00,0.40,233,0x59468516a8259058bad1ca5f8f4bff190d30e066,nftfi,"10,878.59",False,14487198,p2p,6191
1783,0xd46c6a741b174fc8b32667ce8e4fde7e04ac8c5e981901d6cebb07aab65ef905,2022-03-31 04:15:27.000000 UTC,-6304,0xafecd45b9b0a36d9ac54ede54cba07368faa74c6,0x0aff497bd016000185b1c8302fa98a88ff4a4178,0.25,0.27,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-30 04:15:27.000000 UTC,30.00,0.91,7146,0x1a2f71468f656e97c2f86541e57189f59951efe7,nftfi,849.60,False,14491745,p2p,6304
13071,0x282157efbee89719c66db19b9b10185c8df77902066cef1f327918563b7dcaa7,2022-03-31 23:12:35.000000 UTC,-6353,0x152d030f6a7fac2f8239a8c4cac25d76c231dfb3,0x0aff497bd016000185b1c8302fa98a88ff4a4178,1.25,1.33,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-30 23:12:35.000000 UTC,30.00,0.76,51595,0x5cc5b05a8a13e3fbdb0bb9fccd98d38e50f90c38,nftfi,"4,113.98",False,14496818,p2p,6353
13069,0xa72a6a1436a2cc8e28bdbdea2f2dc35f28b31e313538c77cf5d57654b86e5171,2022-03-31 23:12:45.000000 UTC,-6354,0x152d030f6a7fac2f8239a8c4cac25d76c231dfb3,0x0aff497bd016000185b1c8302fa98a88ff4a4178,1.25,1.33,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-30 23:12:45.000

In [15]:
nftfi.head(2)

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z,1
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN,2


In [16]:
print('MQ:\n')
print("Are loan_id unique? Once grouped by unique, the counts are: ")
mq_loan_id_count = pd.DataFrame(mq.groupby('loan_id').agg('count')['transaction_hash'])
mq_loan_id_count = mq_loan_id_count.reset_index()

display(mq_loan_id_count['transaction_hash'].unique())
display(mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1])
duplicated_mq_loans = pd.merge(left=mq, right=mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1], left_on='loan_id', right_on='loan_id')
display(duplicated_mq_loans)
display(duplicated_mq_loans.amt_in_usd.sum())
print("Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(mq.groupby('loan_id_cleaned').agg('count')['transaction_hash'].unique())


MQ:

Are loan_id unique? Once grouped by unique, the counts are: 


array([1, 2])

,loan_id,transaction_hash
177,-6545,2
12386,21523,2
13659,24224,2
16548,30521,2
16549,30522,2
16550,30523,2


,transaction_hash_x,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned,transaction_hash_y
0,0x5d215283e0326ebddc63178230b83a1b00612e9fe30de12c4e52aaea4b2c1879,2023-03-26 16:44:11.000000 UTC,30523,0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06,0x1797b4235473fbe0e7e44322f01c1b5618ebda41,1.00,1.02,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-25 16:44:11.000000 UTC,30.00,0.29,14093,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,"1,764.35",False,16912867,p2p,30523,2
1,0x5d215283e0326ebddc63178230b83a1b00612e9fe30de12c4e52aaea4b2c1879,2023-03-26 16:44:11.000000 UTC,30523,0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06,0x1797b4235473fbe0e7e44322f01c1b5618ebda41,1.00,1.02,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-25 16:44:11.000000 UTC,30.00,0.29,14093,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,"1,764.35",False,16912867,p2p,30523,2
2,0x205c9f50b22b5a4751b14e09efb1eacc757ede76ca421a6b500a0962155ed4e4,2023-03-26 16:43:47.000000 UTC,30522,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:47.000000 UTC,14.00,0.79,365,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912865,p2p,30522,2
3,0x205c9f50b22b5a4751b14e09efb1eacc757ede76ca421a6b500a0962155ed4e4,2023-03-26 16:43:47.000000 UTC,30522,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:47.000000 UTC,14.00,0.79,365,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912865,p2p,30522,2
4,0xd5f969b32332020a8e7129e4aa2f20a5bff446b31e0054469aa47013268704d7,2023-03-26 16:43:11.000000 UTC,30521,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:11.000000 UTC,14.00,0.79,323,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912862,p2p,30521,2
5,0xd5f969b32332020a8e7129e4aa2f20a5bff446b31e0054469aa47013268704d7,2023-03-26 16:43:11.000000 UTC,30521,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:11.000000 UTC,14.00,0.79,323,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912862,p2p,30521,2
6,0xfb83f7b98831eec2595e1af36e064057cfac3018b15513a6cb8dd7017c6c4df8,2023-02-06 20:30:11.000000 UTC,24224,0x910d44c699c24cddd03ce8e7e8cc40835464fb89,0x35fa3c19cf39674353025234ed001bcbb36af35c,0.31,0.32,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-08 20:30:11.000000 UTC,30.00,0.55,32395,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,511.04,False,16572143,p2p,24224,2
7,0xfb83f7b98831eec2595e1af36e064057cfac3018b15513a6cb8dd7017c6c4df8,2023-02-06 20:30:11.000000 UTC,24224,0x910d44c699c24cddd03ce8e7e8cc40835464fb89,0x35fa3c19cf39674353025234ed001bcbb36af35c,0.31,0.32,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-08 20:30:11.000000 UTC,30.00,0.55,32395,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,511.04,False,16572143,p2p,24224,2
8,0xbbd45783e2cd294f86ec604831eeeca7be8bd844818ef7ea52a3df7eb5a01c94,2023-01-18 06:03:35.000000 UTC,21523,0x45d1a3acee4b57f5bb078a14d993364f80e86657,0xebd0db1bfaa50e7abe52412644e968166f349755,0.90,0.92,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-02-01 06:03:35.000000 UTC,14.00,0.69,1662,0x394e3d3044fc89fcdd966d3cb35ac0b32b0cda91,nftfi,"1,428.82",False,16431774,p2p,21523,2
9,0xbbd45783e2cd294f86ec604831eeeca7be8bd844818ef7ea52a3df7eb5a01c94,2023-01-18 06:03:35.000000 UTC,21523,0x45d1a3acee4b57f5bb078a14d993364f80e86657,0xebd0db1bfaa50e7abe52412644e968166f349755,0.90,0.92,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-02-01 06:03:35.000000 U

20059.692436

Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2, 3])

In [17]:
print(f"duplicate lines in MQ dataset {mq.shape[0] - mq.drop_duplicates().shape[0]}")
print(f"duplicate lines in nftfi dataset {nftfi.shape[0] - nftfi.drop_duplicates().shape[0]}")

duplicate lines in MQ dataset 6
duplicate lines in nftfi dataset 1


## Clean raw NFTfi dataset

In [18]:
nftfi['date'] = pd.to_datetime(nftfi['date'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_start_time'] = pd.to_datetime(nftfi['loan_start_time'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_repaid_time'] = pd.to_datetime(nftfi['loan_repaid_time'], format="%Y-%m-%d %H:%M:%S%z")

# Remove invalid values
nftfi.replace(r"#DIV/0!", np.nan, regex=True, inplace=True)
nftfi.replace(r"#N/A", "", regex=True, inplace=True)



# Divide the currency by its respective divider
# now map missing usd_value, e.g. for stablecoin-denominated loans
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

columns_to_clean = ['loan_principal_amount', 'maximum_repayment_amount', 'maximum_repayment_amount']  # Divide loanPrincipalAmount and maximumRepaymentAmount by ETH <> WEI i.e. 1^18
for col in columns_to_clean:
    nftfi[col] = nftfi[col].astype(np.float64)
    nftfi.loc[nftfi['loan_erc20denomination'] == weth_address, col] /= 10 ** 18
    nftfi.loc[nftfi['loan_erc20denomination'] == dai_address, col] /= 10 ** 18
    nftfi.loc[nftfi['loan_erc20denomination'] == usdc_address, col] /= 10 ** 6


# nftfi['no_of_days'] = nftfi['no_of_days'].astype(np.float64)
nftfi = nftfi.rename(columns={'loan_apr': 'apr'})
nftfi['apr'] = nftfi['apr'].astype(np.float64)

# Drop last column as it is unnamed
nftfi = nftfi.drop('', axis=1, errors='ignore')

# set BOOL columns to bool type
cols = ['repaid', 'liquidated']
nftfi['repaid'] = nftfi['repaid'].fillna(False)
nftfi['repaid'] = nftfi['repaid'].replace('', False)
for col in cols:
    # nftfi[col] = nftfi[col].astype('boolean')
    nftfi[col] = nftfi[col].map({'True': True, 'False': False})
    nftfi[col] = nftfi[col].astype(bool)
nftfi['repaid'] = nftfi['repaid'].astype(bool)

In [19]:
nftfi.sort_values(by='loan_principal_amount')

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
34288,2023-04-27 18:22:11+00:00,v2-1.loan.fixed-34288,2023-04-27 18:22:11+00:00,2023-05-27T18:22:11.000Z,True,30.00,True,0.00,0.00,NaN,NaN,2582465753,NaN,0x11127F85B096B694E1b77bfCbe861f5F11f88896,0x7d6E22db7C2Ee44859061061f99E55257A5cEaC1,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,86571878219782098813062260535004457283669633212530069131913859874549537601361,False,31.42,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2023-04-27 18:42:47+00:00,NaN,34288
18116,2022-12-21 04:35:11+00:00,v2-1.loan.fixed-18117,2022-12-21 04:35:11+00:00,2022-12-28T04:35:11.000Z,True,7.00,True,0.00,0.00,NaN,NaN,"1,000,000,000,000.00",NaN,0x83bbee3c8a1dE9D5277a1fACe273c14Ae9aA8aF6,0x04F78cdDeC5a6cf4c0B40fBeBEB5cA49DebD44B5,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,33266523340797296217853586896260878768936399018396274840948193876064791248822,True,521.43,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaT,NaN,18117
33946,2023-04-25 16:26:11+00:00,v2-1.loan.fixed-33946,2023-04-25 16:26:11+00:00,2023-04-26T16:26:11.000Z,True,1.00,True,0.00,0.00,NaN,NaN,1000000000000,NaN,0xBea320E6a570905cE583446589DeaBAe65Cd7A93,0xf52a65497128702bf2d469600FEa18858235Ff82,0x1792a96E5668ad7C167ab804a100ce42395Ce54D,3705,False,456.25,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2023-04-25 16:29:11+00:00,NaN,33946
13405,2022-10-18 12:36:59+00:00,v2-1.loan.fixed-13406,2022-10-18 12:36:59+00:00,2022-10-19T12:36:59.000Z,True,1.00,True,0.00,0.00,NaN,NaN,"1,000,000,000,000.00",NaN,0xb88E4282e7CF05B56579EA57DAE3cA570150A23b,0x8a32121D737Ce9c7b7b6e17cC7F10d7C2D5F8adC,0xF3E778F839934fC819cFA1040AabaCeCBA01e049,5456,False,365.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaT,2022-10-20T08:02:59.000Z,13406
15235,2022-11-09 13:31:23+00:00,v2.loan.fixed.collection-15236,2022-11-09 13:31:23+00:00,2022-11-10T13:31:23.000Z,True,1.00,True,0.00,0.00,NaN,NaN,"1,000,000,000,000.00",NaN,0x8a32121D737Ce9c7b7b6e17cC7F10d7C2D5F8adC,0xb88E4282e7CF05B56579EA57DAE3cA570150A23b,0xF3E778F839934fC819cFA1040AabaCeCBA01e049,5446,False,365.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-11-09 13:38:11+00:00,NaN,15236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2022-04-19 21:43:38+00:00,v2.loan.fixed-728,2022-04-19 21:43:38+00:00,2022-07-18T21:43:38.000Z,True,90.00,True,"200,000.00",0.00,NaN,NaN,"24,657,534,247,000,000,692,224.00",NaN,0x2fdf502ac2f054d5983A4a4ba916B2F2d4056685,0x864749824EA21C64169433e4f4EE4B1F4BF76B99,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,3673,False,50.00,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-05-01 15:11:00+00:00,NaN,728
34526,2023-04-30 02:27:11+00:00,v2-1.loan.fixed-34526,2023-04-30 02:27:11+00:00,2023-05-02T02:27:11.000Z,True,2.00,True,"271,000.00",0.27,NaN,NaN,742465753,NaN,0x5CAdce51dB28AB261E50743fEB881a3F7AcA0A69,0xA9c501101C7B165090ABb56E95efe1129Df9DEAC,0x46C9CFB32627B74F91e0B5ad575c247AEc7e7847,62,False,50.00,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,2023-05-01 01:42:35+00:00,NaN,34526
9851,2022-08-29 10:24:57+00:00,v2.loan.fixed-9852,2022-08-29 10:24:57+00:00,2022-09-22T10:24:57.000Z,True,24.00,True,"350,000.00",0.00,NaN,NaN,"999,999,999,999,955,959,808.00",NaN,0xF4Fb9FA23edB32215E5284cf7dBfDB5607d51a5b,0x39fEc2e2beaB6a63c1E763D0dc4120AF60BEe39F,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,441,False,4.35,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-09-20 00:48:47+00:00,NaN,9852
27018,2023-02-27 14:27:47+00:00,v2-1.loan.fixed-27019,2023-02-27 14:27:47+00:00,2023-11-25T14:27:47.000Z,True,271.00,True,"500,000.00",0.00,NaN,NaN,"37,123,287,670,999,977,295,872.00",NaN,0xB82ea9DFce1AE280211f2991a09F77Fa06bbd391,0x41797b5e0398aF475422f6F33F2dc81d9a24aE33,0x9a129032F01EB4dDD764c1777c81b771C34a2fbE,40,True,10.00,0x6B17547

## Enrich datasets
### Enrich NFTfi dataset with ETH price and compute usd_value of loan for weth-denominated loans

In [20]:
# Enrich dataset with ETHUSD rate i.e. ETHPrice, then fill USDValue for USDValue from loanERC20Denomination == weth_address
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'.lower()
price_df = pd.read_csv('../../analytics_bot_langchain/data/nftfi/eth_usd_input_data.csv')
price_df = price_df.rename(columns={'datetime': 'date'})
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d %H:%M:%S%z')
price_df = price_df.drop(columns=['open', 'low', 'volume'])

nftfi['date'] = pd.to_datetime(nftfi['date'])
nftfi['loan_principal_amount'] = nftfi['loan_principal_amount'].astype(float)

nftfi = nftfi.sort_values(by='date')
price_df = price_df.sort_values(by='date')

nftfi = pd.merge_asof(nftfi, price_df, on='date', direction='backward')
nftfi['eth_price'] = nftfi['close']
nftfi = nftfi.drop(columns=['close'])
nftfi['loan_erc20denomination'] = nftfi['loan_erc20denomination'].astype(str).str.lower().str.strip()
nftfi.loc[nftfi['loan_erc20denomination'] == weth_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['loan_principal_amount'] * nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['eth_price']

# now map missing usd_value, e.g. for stablecoin-denominated loans
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
stablecoin_addresses = [usdc_address, usdt_address, dai_address]
for stablecoin_address in stablecoin_addresses:
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address]['loan_principal_amount']



In [21]:
print('\n\nNFTfi: \n')
print("Are loan_no unique? Once grouped by unique, the counts are: ")
nftfi_loan_no_count = pd.DataFrame(nftfi.groupby('loan_no').agg('count')['borrower'])
duplicated_nftfi_loans = pd.merge(left=nftfi, right=nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1], left_on='loan_no', right_on='loan_no')
display(duplicated_nftfi_loans)
display(duplicated_nftfi_loans.usd_value.sum())

display(nftfi_loan_no_count['borrower'].unique())
display(nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1])

print("Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(nftfi.groupby('loan_no_cleaned').agg('count')['borrower'].unique())



NFTfi: 

Are loan_no unique? Once grouped by unique, the counts are: 


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower_x,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned,high,eth_price,usd_value,borrower_y
0,2023-03-14 14:19:59+00:00,v2-1.loan.fixed-29040,2023-03-14 14:19:59+00:00,2023-04-13T14:19:59.000Z,True,30.00,True,0.35,0.00,NaN,NaN,"22,726,000,000,000,000.00",NaN,0x34a1E2E6b781763001ef7187Ba26ACb383BAed91,0xBDeD327A145B4703300E10204E6d180e1a15C844,0x942BC2d3e7a589FE5bd4A5C6eF9727DFd82F5C8a,6422,True,79.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaT,NaN,29040,"1,740.72","1,733.65",606.78,2
1,2023-03-14 14:19:59+00:00,v2-1.loan.fixed-29040,2023-03-14 14:19:59+00:00,2023-04-13T14:19:59.000Z,True,30.00,True,0.35,0.00,NaN,NaN,"22,726,000,000,000,000.00",NaN,0x34a1E2E6b781763001ef7187Ba26ACb383BAed91,0xBDeD327A145B4703300E10204E6d180e1a15C844,0x942BC2d3e7a589FE5bd4A5C6eF9727DFd82F5C8a,6422,True,79.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaT,NaN,29040,"1,740.72","1,733.65",606.78,2


1213.555

array([1, 2])

,borrower
loan_no,
v2-1.loan.fixed-29040,2


Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2])

### Enrich nftfi table with ERC20 denominations

In [22]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [23]:
# Confirm that the above addresses matches the one available in MQ dataset:
mq.erc20_address.unique()

array(['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       '0x6b175474e89094c44da98b954eedeac495271d0f',
       '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], dtype=object)

#### Enrich NFTfi table with loan types

In [24]:
nftfi['loan_type'] = nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0])
nftfi['loan_type'].unique()

array(['v2.loan.fixed', 'v2-1.loan.fixed', 'v2.loan.fixed.collection'],
      dtype=object)

In [25]:
mq

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p,36220
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p,36212
12757,0x0bf87b1d9ecec93f80c864f41899cb206c261c30adad5999318fd42c7ef294f8,2023-05-18 10:16:23.000000 UTC,36207,0xb04a68660362b3c10a6745b14d383a27503c3d50,0x9cd074f0192bb2a477da3271ba03db4fc5cd38c7,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 10:16:23.000000 UTC,30.00,0.50,8231,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,nftfi,"1,095.78",False,17285778,p2p,36207
16031,0xf660371bc68bfdcf8810846fa8b418147f4091081fba11c0cfdefa5ddcef8fd4,2023-05-18 09:23:11.000000 UTC,36204,0xc78c3f6cec7d593e0fd931670ca820aac4eee08c,0x5b20dce335d131c6de0cc11b481ace2aade788df,1.30,1.37,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 09:23:11.000000 UTC,30.00,0.70,3000118,0x64780ce53f6e966e18a22af13a2f97369580ec11,nftfi,"2,374.36",False,17285519,p2p,36204
17508,0x1bc701828c5f3b8e1e753ae095cd0487e9f431545c816fbe27110a639de53ba7,2023-05-18 08:51:47.000000 UTC,36201,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 08:51:47.000000 UTC,9.00,0.36,12015,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,740.40",False,17285365,p2p,36201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12520,0x504e78de6e5a1aa3b3c54f4a681a6f7866ba9c827ff04cabd4c7d986bde13dc7,2022-03-30 11:03:13.000000 UTC,-6191,0x8854631995b99bbfca16b8ea339538950181132a,0x85db67812169378666547e03b293785d7d9a6b06,3.20,3.31,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:03:13.000000 UTC,30.00,0.40,233,0x59468516a8259058bad1ca5f8f4bff190d30e066,nftfi,"10,878.59",False,14487198,p2p,6191
584,0x7d7682a9827b44571da3bdc1f7342d9ad3c985e7aedba428bbf3a2f506306cb6,2022-03-30 10:43:05.000000 UTC,3,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-03-30 10:50:17.000000 UTC,0.01,0.73,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,nftfi,33.89,False,14487122,p2p,3
745,0x9d783e3bbde1663ffa2dcc0bf5eabeccbe3f58475938c42e52f792f8eb85eddf,2022-03-30 10:36:00.000000 UTC,2,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:36:00.000000 UTC,14.00,0.10,735355,0x06012c8cf97bead5deae237070f9587f8e7a266d,nftfi,33.89,False,14487091,p2p,2
7527,0xbd19dbb314cdcc2be0c0c579e639596635e77b7d5e78062e3d14ae27afa12266,2022-03-30 10:24:07.000000 UTC,-6190,0x52ac5997d2ac830d54b4bad431ab95546d279144,0x25f9ee5543d3d815051fcbedffd283ce457f7426,"1,500.00","1,570.00",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-04-29 10:24:07.000000 UTC,30.00,0.57,462,0x3fe1a4c1481c8351e91b64d5c398b159de07cbc5,nftfi,"1,500.00",False,14487034,p2p,6190


In [26]:
# enrich dataset with collection name, blockchain
try:
    collection_names_df = pd.read_csv('analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
except FileNotFoundError:
    collection_names_df = pd.read_csv('../../analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
collection_names_df = collection_names_df.rename(columns={'contract_address': 'collection_address', 'name': 'collection_name'})
collection_names_df['collection_address'] = collection_names_df['collection_address'].astype(str).str.lower().str.strip()
collection_names_df = collection_names_df.drop(columns=['symbol', 'standard', 'blockchain'])
display(collection_names_df.head(2))
mq['collection_address'] = mq['collection_address'].astype(str).str.lower().str.strip()

mq_existing_cols = mq.columns
mq = pd.merge(left=mq, right=collection_names_df, on='collection_address', how='left')
new_columns_order = ['collection_name'] + list(mq_existing_cols)
mq = mq[new_columns_order]

,collection_address,collection_name
0,0x3b37270c332b5c6cf6aac0103d9d896f5dcafb1d,Riia the Girls
1,0x668d40fb53871aa139ae306bca88f00bb8c475fb,Simpin Santa


## Check USD value discrepancies for matching loans on transaction hash

In [27]:
display(nftfi.head(2))
display(mq.head(2))

mq['loan_id'] = mq['loan_id'].astype(int)
nftfi['loan_no_cleaned'] = nftfi['loan_no_cleaned'].astype(int)


nftfi.columns = [f'nftfi_{col}' for col in nftfi.columns]
mq.columns = [f'mq_{col}' for col in mq.columns]
full_inner = pd.merge(nftfi, mq, how='inner', left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id', indicator=True)


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned,high,eth_price,usd_value,loan_erc20denomination_name,loan_type
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30 10:21:32+00:00,2022-04-13T10:21:32.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:27:49+00:00,2022-03-13T10:48:13.000Z,1,"3,396.32","3,394.93",33.95,weth,v2.loan.fixed
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30 10:36:00+00:00,2022-04-13T10:36:00.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:36:56+00:00,NaN,2,"3,398.36","3,397.60",33.98,weth,v2.loan.fixed


,collection_name,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
0,Meebits,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p,36220
1,NFTfi Locked Bundle,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p,36212


In [28]:
usd_diff = full_inner[['mq_amt_in_usd','nftfi_usd_value', 'nftfi_loan_no_cleaned']]# .dropna()
display(usd_diff.head(2))
display(usd_diff.tail(2))
usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']
display(usd_diff.tail(2))

,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned
0,33.89,33.95,1
1,33.89,33.98,2


,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned
19219,858.33,858.38,36212
19220,"2,738.22","2,739.49",36220


/tmp/ipykernel_709936/4014243634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']


,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned,usd_diff_nftfi_minus_mq
19219,858.33,858.38,36212,0.04
19220,"2,738.22","2,739.49",36220,1.28


In [29]:
display(pd.DataFrame(usd_diff.min().values).T)
# nftfi.loc[nftfi['nftfi_transaction_hash']=='0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210'].T
# display(pd.DataFrame(usd_diff.min().T['nftfi_transaction_hash'])

,0,1,2,3
0,0.00,0.00,1.00,"-2,380.57"


In [30]:
pd.DataFrame(usd_diff.usd_diff_nftfi_minus_mq.describe())

,usd_diff_nftfi_minus_mq
count,"19,221.00"
mean,3.15
std,95.57
min,"-2,380.57"
25%,-3.47
50%,0.00
75%,3.75
max,"4,598.59"


# Check per collection borrow volume: all time, monthly, daily etc

In [31]:
start = '2020-04-01'
end = '2024-05-01'

april_mq = mq.loc[mq['mq_block_timestamp'] >= start].loc[mq['mq_block_timestamp'] < end]
display(april_mq.shape)
april_nftfi = nftfi.loc[nftfi['nftfi_date'] >= start].loc[nftfi['nftfi_date'] < end]
display(april_nftfi.shape)


april_full_outer = pd.merge(april_nftfi, april_mq, how='outer', left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id', indicator=True)

april_full_outer_without_nftfi_na = april_full_outer.dropna(subset=['nftfi_usd_value'])
april_full_outer_without_nftfi_na.head(2)

(19547, 21)

(36221, 28)

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30 10:21:32+00:00,2022-04-13T10:21:32.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:27:49+00:00,2022-03-13T10:48:13.000Z,1.00,"3,396.32","3,394.93",33.95,weth,v2.loan.fixed,Art Blocks,0x166d57a868ae6c3e34c192546aedc243eda0c7ba6685b741f655e3329e2b6e29,2022-03-30 10:21:32.000000 UTC,1.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:21:32.000000 UTC,14.00,0.10,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,nftfi,33.89,False,"14,487,024.00",p2p,1,both
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30 10:36:00+00:00,2022-04-13T10:36:00.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:36:56+00:00,NaN,2.00,"3,398.36","3,397.60",33.98,weth,v2.loan.fixed,NaN,0x9d783e3bbde1663ffa2dcc0bf5eabeccbe3f58475938c42e52f792f8eb85eddf,2022-03-30 10:36:00.000000 UTC,2.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:36:00.000000 UTC,14.00,0.10,735355,0x06012c8cf97bead5deae237070f9587f8e7a266d,nftfi,33.89,False,"14,487,091.00",p2p,2,both


In [32]:
april_full_outer_without_nftfi_na = april_full_outer_without_nftfi_na.groupby('mq_collection_name').agg(sum)[['nftfi_usd_value', 'mq_amt_in_usd']]
april_full_outer_without_nftfi_na['nftfi_minus_mq'] = april_full_outer_without_nftfi_na['nftfi_usd_value'] - april_full_outer_without_nftfi_na['mq_amt_in_usd']
april_full_outer_without_nftfi_na['abs_nftfi_minus_mq'] = april_full_outer_without_nftfi_na['nftfi_minus_mq'].abs()
april_full_outer_without_nftfi_na.head(5)

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
0N1 Force,"94,691.19","94,674.63",16.56,16.56
0xmons,"124,984.77","125,199.34",-214.57,214.57
10KTF,355.15,354.46,0.70,0.70
10KTF Gucci Grail,"14,453.46","14,454.72",-1.25,1.25
6529 Gradient,"16,827.47","16,800.96",26.51,26.51


In [33]:
april_full_outer_without_nftfi_na.describe()

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
count,161.00,161.00,161.00,161.00
mean,"639,622.44","639,245.24",377.20,516.76
std,"2,651,025.30","2,649,606.57","1,922.36","1,889.43"
min,9.55,9.52,"-2,014.51",0.00
25%,"4,635.16","4,501.37",-5.54,5.22
50%,"35,071.80","35,108.90",2.98,26.69
75%,"203,940.15","203,905.44",69.22,133.80
max,"25,879,304.02","25,864,865.57","14,438.45","14,438.45"


In [34]:
pd.options.plotting.backend = "plotly"
april_full_outer_without_nftfi_na.plot(x=april_full_outer_without_nftfi_na.index, y=april_full_outer_without_nftfi_na.nftfi_minus_mq)

In [35]:
april_full_outer_without_nftfi_na.sort_values(by='abs_nftfi_minus_mq', ascending=False)

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
MutantApeYachtClub,"25,879,304.02","25,864,865.57","14,438.45","14,438.45"
Art Blocks,"7,068,292.74","7,054,803.80","13,488.94","13,488.94"
Otherdeed for Otherside,"11,345,645.83","11,336,490.19","9,155.64","9,155.64"
Moonbirds,"6,000,843.99","5,993,616.90","7,227.09","7,227.09"
PROOF Collective,"669,930.00","663,021.03","6,908.97","6,908.97"
Cool Cats,"1,072,619.91","1,069,313.39","3,306.52","3,306.52"
DeadFellaz,"425,886.70","427,901.21","-2,014.51","2,014.51"
Doodles,"14,151,584.81","14,149,985.02","1,599.79","1,599.79"
IKB Cachet de Garantie,"73,683.22","75,235.17","-1,551.95","1,551.95"


In [36]:

april_full_outer_without_nftfi_na.sort_values(by='nftfi_minus_mq')

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
DeadFellaz,"425,886.70","427,901.21","-2,014.51","2,014.51"
IKB Cachet de Garantie,"73,683.22","75,235.17","-1,551.95","1,551.95"
CloneX,"8,255,867.10","8,257,267.54","-1,400.45","1,400.45"
Land,"736,552.60","737,649.64","-1,097.03","1,097.03"
Meebits,"1,456,624.58","1,457,605.42",-980.84,980.84
The Captainz,"1,531,409.15","1,532,337.41",-928.26,928.26
FVCK_CRYSTAL//,"51,090.41","51,637.48",-547.07,547.07
TRAITORS open edition by XCOPY,"71,056.82","71,578.84",-522.01,522.01
Creepz by OVERLORD,"395,516.66","395,963.52",-446.86,446.86


In [37]:
collection_of_interest = 'MutantApeYachtClub'
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].describe())
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].head(3))


,nftfi_no_of_days,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest_in_usd,nftfi_apr,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,mq_loan_id,mq_principal_amount,mq_repayment_amount,mq_duration_in_days,mq_apr,mq_amt_in_usd,mq_block_number
count,"1,169.00","1,169.00","1,169.00",0.00,0.00,0.00,"1,169.00","1,169.00","1,169.00","1,169.00","1,169.00","1,176.00","1,176.00","1,176.00","1,176.00","1,176.00","1,176.00","1,176.00"
mean,31.16,689.30,0.00,NaN,NaN,NaN,42.10,"15,062.57","1,744.31","1,742.32","22,137.98","14,932.64",774.00,818.06,31.14,0.42,"22,266.66","15,802,923.90"
std,25.61,"3,803.29",0.00,NaN,NaN,NaN,27.14,"10,598.59",552.10,551.86,"16,125.68","10,699.67","4,232.84","4,531.10",25.54,0.27,"16,245.15","839,542.05"
min,1.00,1.00,0.00,NaN,NaN,NaN,2.00,19.00,924.32,909.95,"1,612.06","-6,918.00",1.00,1.00,1.00,0.02,"1,609.33","14,503,242.00"
25%,14.00,9.00,0.00,NaN,NaN,NaN,27.26,"5,351.00","1,310.15","1,309.18","13,491.40","5,293.75",9.00,9.43,14.00,0.28,"13,512.49","14,972,042.50"
50%,30.00,11.10,0.00,NaN,NaN,NaN,38.50,"14,104.00","1,614.69","1,612.36","16,686.70","13,933.00",11.15,11.70,30.00,0.38,"16,710.27","15,831,439.00"
75%,30.00,14.00,0.00,NaN,NaN,NaN,49.99,"22,378.00","1,878.16","1,878.16","21,533.98","22,338.25",14.00,14.50,30.00,0.50,"21,610.58","16,472,865.00"
max,180.00,"48,374.81",0.03,NaN,NaN,NaN,299.00,"36,188.00","3,525.92","3,525.92","90,323.20","36,188.00","50,000.00","54,808.22",180.00,2.99,"90,195.42","17,284,673.00"


,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge
18,2022-04-05 09:22:47+00:00,v2.loan.fixed-19,2022-04-05 09:22:47+00:00,2022-05-05T09:22:47.000Z,True,30.00,True,16.00,0.00,NaN,NaN,"1,387,266,000,000,000,000.00",NaN,0x0AFF497Bd016000185b1c8302fA98a88fF4A4178,0xf8e8dEda67cFB656cC067A42E170B5eA852111D5,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,9747,False,105.49,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-04-29 19:08:31+00:00,NaN,19.00,"3,525.92","3,525.92","56,414.72",weth,v2.loan.fixed,MutantApeYachtClub,0x7c30f78bb0ac1e55e747f9320c4797cb5b9a5a0dcf78e50f8e67d5324d1f2956,2022-04-05 09:22:47.000000 UTC,19.00,0xf8e8deda67cfb656cc067a42e170b5ea852111d5,0x0aff497bd016000185b1c8302fa98a88ff4a4178,16.00,17.39,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-05-05 09:22:47.000000 UTC,30.00,1.05,9747,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"56,408.13",False,"14,525,066.00",p2p,19,both
24,2022-04-05 11:05:57+00:00,v2.loan.fixed-25,2022-04-05 11:05:57+00:00,2022-05-05T11:05:57.000Z,True,30.00,True,16.00,0.00,NaN,NaN,"986,301,000,000,000,000.00",NaN,0x0AFF497Bd016000185b1c8302fA98a88fF4A4178,0x1Ca4B7B0901Ff8e802B45575E79b32B69cDbf95B,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,15016,False,75.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-04-14 11:35:26+00:00,NaN,25.00,"3,519.18","3,518.79","56,300.64",weth,v2.loan.fixed,MutantApeYachtClub,0xdf586712086558b928c9b666e4a10c7c3eecdc1c47a8497eabd7a004d26d472d,2022-04-05 11:05:57.000000 UTC,25.00,0x1ca4b7b0901ff8e802b45575e79b32b69cdbf95b,0x0aff497bd016000185b1c8302fa98a88ff4a4178,16.00,16.99,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-05-05 11:05:57.000000 UTC,30.00,0.75,15016,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"56,274.07",False,"14,525,522.00",p2p,25,both
30,2022-04-05 23:34:07+00:00,v2.loan.fixed-31,2022-04-05 23:34:07+00:00,2022-07-04T23:34:07.000Z,True,90.00,True,19.50,0.00,NaN,NaN,"2,307,945,000,000,000,000.00",NaN,0x1254958BD5073C6B238E516298f0c48f6f60A78e,0x224b3F081075c1bC10e8FCE9C03719214F714e23,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,5464,False,48.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-04-06 00:42:59+00:00,NaN,31.00,"3,437.77","3,436.73","67,016.24",weth,v2.loan.fixed,MutantApeYachtClub,0x8782df003fe495ca6ca6f3f2b7b0695fc7c52f3148ebc818afea08c0be038889,2022-04-05 23:34:07.000000 UTC,31.00,0x224b3f081075c1bc10e8fce9c03719214f714e23,0x1254958bd5073c6b238e516298f0c48f6f60a78e,19.50,21.81,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-07-04 23:34:07.000000 UTC,90.00,0.48,5464,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"67,161.92",False,"14,528,908.00",p2p,31,both


In [38]:
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']['_merge'].unique())
april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']

['right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge
36229,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0x6ebe02eb92356ecf1439fb880b63ce6aaed73fb18a014a31803a6455aecd62a9,2022-06-11 15:44:26.000000 UTC,"-6,918.00",0x1df428833f2c9fb1ef098754e5d710432450d706,0xd07cbcb30a8dca329ca8f8298f4428355d4385bc,"14,365.00","14,612.94",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-06-18 15:44:26.000000 UTC,7.00,0.90,6331,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"14,365.00",False,"14,945,087.00",p2p,6918,right_only
36231,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0x7f337e68b046da338c27ecdcd5c56942134877cae9cfbb1358496313416b1cf4,2022-05-10 22:46:39.000000 UTC,"-6,906.00",0x5641ae487adcafe5bc50fca8110b4df69b31c6e3,0x8e3d6c8f591d73ebcf79810ebc4b3adf829d2fb6,"40,000.00","41,249.32",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-06-09 22:46:39.000000 UTC,30.00,0.38,15140,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"40,000.00",False,"14,751,252.00",p2p,6906,right_only
36238,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0x8a3762f8257af05d5bd4d20039577115b39b639c89e47a196bd186d3c2f5fe0f,2022-05-05 05:47:49.000000 UTC,"-6,898.00",0x1430cc22bccfece3d894e298a3845d3ac8e871f9,0x768f2a7ccdfde9ebdfd5cea8b635dd590cb3a3f1,13.00,13.38,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-06-04 05:47:49.000000 UTC,30.00,0.36,8196,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"38,286.85",False,"14,715,501.00",p2p,6898,right_only
36241,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0x6595205584132394f1278f67ca1ccbde8e50f55342a79ba0d3b8669b20ed24e4,2022-05-02 10:36:27.000000 UTC,"-6,891.00",0x391e3a871e75e041fc16c70d8204ef7c6e327401,0xd4e26683635bf3dc9ead5f31b935c33cc1ce1838,25.00,25.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-06-01 10:36:27.000000 UTC,30.00,0.30,2979,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"70,251.95",False,"14,697,815.00",p2p,6891,right_only
36246,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0xdab281b1e180e4b7db32b60b36a0be5f2d18073e7701635e6a21bbc450c13ec9,2022-04-29 05:14:39.000000 UTC,"-6,885.00",0xf46c246c69f547bc3b864648bf13324c355e519b,0xd4e26683635bf3dc9ead5f31b935c33cc1ce1838,25.00,25.72,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-05-29 05:14:39.000000 UTC,30.00,0.35,8015,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"72,986.66",False,"14,677,280.00",p2p,6885,right_only
36466,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MutantApeYachtClub,0x5406126dec566ca45947a9ef2bd418d6f71db015cdb895c9353d2323b0458fa0,2022-04-02 02:29:20.000000 UTC,"-6,441.00",0x1df4ac78f2a1886a834c36c24c77ac18e4ba269e,0x0aff497bd016000185b1c8302fa98a88ff4a4178,10.00,10.35,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756

In [39]:
# TODO: compute the implied ETH price 
april_full_outer_for_collection = april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest]
april_full_outer_for_collection['nftfi_minus_mq'] = april_full_outer_for_collection['nftfi_usd_value'] - april_full_outer_for_collection['mq_amt_in_usd']
pd.DataFrame(april_full_outer_for_collection['nftfi_minus_mq'].describe()).T

/tmp/ipykernel_709936/3243541687.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,count,mean,std,min,25%,50%,75%,max
nftfi_minus_mq,"1,169.00",12.35,161.85,"-1,082.11",-27.97,0.14,41.55,"1,412.01"


In [48]:
nftfi.dtypes

nftfi_date                           datetime64[ns, UTC]
nftfi_loan_no                                     object
nftfi_loan_start_time                datetime64[ns, UTC]
nftfi_loan_due_time                               object
nftfi_repaid                                        bool
nftfi_no_of_days                                 float64
nftfi_liquidated                                    bool
nftfi_loan_principal_amount                      float64
nftfi_maximum_repayment_amount                   float64
nftfi_ethprice                                   float64
nftfi_usdvalue                                   float64
nftfi_interest                                    object
nftfi_interest_in_usd                            float64
nftfi_lender                                      object
nftfi_borrower                                    object
nftfi_nft_collateral_contract                     object
nftfi_nft_collateral_id                           object
nftfi_active                   

In [49]:
april_full_outer_for_collection.dtypes
# TODO: loc for wETH address
april_full_outer_for_collection['mq_implied_eth_price'] = april_full_outer_for_collection['mq_amt_in_usd'] / april_full_outer_for_collection['mq_principal_amount']
april_full_outer_for_collection['nftfi_mq_eth_price_diff'] = april_full_outer_for_collection['nftfi_eth_price'] - april_full_outer_for_collection['mq_implied_eth_price']
highest_discrepancy_transactions = april_full_outer_for_collection[['mq_collection_name','nftfi_loan_no_cleaned','mq_transaction_hash','nftfi_minus_mq',
                                                                     'nftfi_loan_erc20denomination_name', 'nftfi_loan_principal_amount',
                                                                       'nftfi_eth_price','mq_implied_eth_price', 'nftfi_mq_eth_price_diff']].sort_values('nftfi_minus_mq')
highest_discrepancy_transactions.head(50)

/tmp/ipykernel_709936/3954958272.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_709936/3954958272.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mq_collection_name,nftfi_loan_no_cleaned,mq_transaction_hash,nftfi_minus_mq,nftfi_loan_erc20denomination_name,nftfi_loan_principal_amount,nftfi_eth_price,mq_implied_eth_price,nftfi_mq_eth_price_diff
15130,MutantApeYachtClub,"15,131.00",0x3328bedcdfd0241d43b7b71e8cbb32a0089e58b95a7093e77c5dfd6a3d9c8279,"-1,082.11",weth,11.00,"1,447.41","1,545.78",-98.37
63,MutantApeYachtClub,64.00,0xfebec5ed4308e5218b6e328db3bcb448fd19f3045f9ea5d5162b4bd1fd35ce09,-771.55,weth,20.00,"3,190.87","3,229.45",-38.58
2911,MutantApeYachtClub,"2,912.00",0x756d398f73b66024cc93abaf99948d482214a973ca73b21c34e8f229dc9d7fa7,-741.25,weth,13.20,"1,835.05","1,891.21",-56.16
4302,MutantApeYachtClub,"4,303.00",0xcdc09b739252a9e04f22ed8159b87436d0886e893a061cf384728ff4d30fb52f,-675.84,weth,11.10,"1,901.68","1,962.57",-60.89
1367,MutantApeYachtClub,"1,368.00",0x887c2f8b99c88caaa4d76d60238e034dd9f1e4361c6ffef1915d5fbe7875ba4b,-671.80,weth,25.50,"2,855.75","2,882.09",-26.34
115,MutantApeYachtClub,116.00,0xc89f682bc648297f176a3930b4a6e8d2ce7c15f60ea25d86706684c53c84a5fb,-568.79,weth,20.00,"3,238.80","3,267.24",-28.44
15244,MutantApeYachtClub,"15,245.00",0xabb0571d54c6d13ab861df0c375376ebbaa03ab617df5301b31746f126810b53,-560.29,weth,10.00,"1,168.10","1,224.13",-56.03
3225,MutantApeYachtClub,"3,226.00",0xd50b70bcca9f3e6ae94aadb4dd45f559048b29d193146794cd519b08dfdf9ef1,-558.86,weth,14.00,"2,015.80","2,055.72",-39.92
1451,MutantApeYachtClub,"1,452.00",0xb5488da38feb58a7da274291207e70cc1262a590944e6393111c468df43918cc,-486.29,weth,25.00,"2,862.01","2,881.46",-19.45
1679,MutantApeYachtClub,"1,680.00",0x72d196bb916175176f8bc1c4a5eb6b99d72020fe8846039d4fe65367fbcb0b48,-463.20,weth,24.50,"2,887.26","2,906.17",-18.91


In [56]:
pd.DataFrame(highest_discrepancy_transactions['nftfi_minus_mq'].cumsum()).head(30)

,nftfi_minus_mq
15130,"-1,082.11"
63,"-1,853.66"
2911,"-2,594.91"
4302,"-3,270.75"
1367,"-3,942.55"
115,"-4,511.34"
15244,"-5,071.63"
3225,"-5,630.49"
1451,"-6,116.77"
1679,"-6,579.98"


In [50]:
# april_full_outer_for_collection.loc[april_full_outer_for_collection['mq_transaction_hash'] == '0xefd0018a270e01f4a53e0687a9791d512cbfcd723a6198608d689451a5111faa']
tsx_with_highest_discrepancy = highest_discrepancy_transactions.iloc[0]['mq_transaction_hash']
max_discrepancy_tsx = april_full_outer_for_collection.loc[april_full_outer_for_collection['mq_transaction_hash'] == tsx_with_highest_discrepancy]
max_discrepancy_tsx

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge,nftfi_minus_mq,mq_implied_eth_price,nftfi_mq_eth_price_diff
15130,2022-11-08 17:57:59+00:00,v2-1.loan.fixed-15131,2022-11-08 17:57:59+00:00,2022-12-08T17:57:59.000Z,True,30.00,True,11.00,0.00,NaN,NaN,"488,219,000,000,000,000.00",NaN,0x8d4169929C0c73A93684B23dC8b7A72D48F2FC58,0x3679AaC6fFe31058bc24DAcE86853F166a74DFDD,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,4427,False,54.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-11-09 02:38:59+00:00,NaN,"15,131.00","1,468.76","1,447.41","15,921.51",weth,v2-1.loan.fixed,MutantApeYachtClub,0x3328bedcdfd0241d43b7b71e8cbb32a0089e58b95a7093e77c5dfd6a3d9c8279,2022-11-08 17:57:59.000000 UTC,"15,131.00",0x3679aac6ffe31058bc24dace86853f166a74dfdd,0x8d4169929c0c73a93684b23dc8b7a72d48f2fc58,11.00,11.49,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-12-08 17:57:59.000000 UTC,30.00,0.54,4427,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"17,003.62",False,"15,926,957.00",p2p,15131,both,"-1,082.11","1,545.78",-98.37


In [51]:
raw_nftfi.loc[raw_nftfi['loanNo'] == max_discrepancy_tsx['nftfi_loan_no'].values[0]]

,date,loanNo,loanStartTime,loanDueTime,repaid,noOfDays,liquidated,loanPrincipalAmount,maximumRepaymentAmount,ETHPrice,USDValue,interest,interestInUSD,lender,borrower,nftCollateralContract,nftCollateralId,active,APR,loanERC20Denomination,loanRepaidTime,loanLiquidationTime
15130,2022-11-08 17:57:59+00:00,v2-1.loan.fixed-15131,2022-11-08T17:57:59.000Z,2022-12-08T17:57:59.000Z,True,30.00,False,11000000000000000000,11488219000000000000,NaN,NaN,"488,219,000,000,000,000.00",NaN,0x8d4169929C0c73A93684B23dC8b7A72D48F2FC58,0x3679AaC6fFe31058bc24DAcE86853F166a74DFDD,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,4427,False,54.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-11-09T02:38:59.000Z,NaN


## Compute April subset for each table

In [43]:
april_mq = mq.loc[mq['block_timestamp'] >= '2023-04-01'].loc[mq['block_timestamp'] < '2023-05-01']
april_mq.shape

KeyError: 'block_timestamp'

In [ ]:
april_nftfi = nftfi.loc[nftfi['date'] >= '2023-04-01'].loc[nftfi['date'] < '2023-05-01']
april_nftfi.shape

#### Check for duplicates

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].head(5))
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].shape[0])

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.head(3).sort_values('loan_id_cleaned', ascending=False))
display(april_nftfi.head(3).sort_values('loan_no_cleaned', ascending=False))

In [ ]:
# Determine the set of unique IDs present in MQ and not in NFTfi
len(set(list(april_mq.loan_id_cleaned)) - set(list(april_nftfi.loan_no_cleaned)))

In [ ]:
# Determine the set of unique IDs present in NFTfi and not in MQ's
len(set(list(april_nftfi.loan_no_cleaned)) - set(list(april_mq.loan_id_cleaned)))

In [ ]:
april_mq.shape, april_nftfi.shape

In [ ]:
april_nftfi.shape[0] - april_mq.shape[0]

## Compute which Loans are available in NFTfi's and not in MQ's, and the converse, by matching on transaction hashes

In [ ]:
# how do i get all rows in df1 that are not in df2
# and all rows in df2 that are not in df1?

# Perform a full outer join on df1 and df2
full_outer = pd.merge(april_nftfi, april_mq, how='outer', on='transaction_hash', indicator=True)

# Filter out the rows that are unique to df1
unique_to_april_nftfi = full_outer[full_outer['_merge'] == 'left_only']

# Filter out the rows that are unique to df2
unique_to_april_mq = full_outer[full_outer['_merge'] == 'right_only']


In [ ]:
print(f"number of loans unique to NFTfi relative to MQ in April 2023: {unique_to_april_nftfi.shape[0]}, i.e. {100 * round(unique_to_april_nftfi.shape[0] / april_nftfi.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")
display(unique_to_april_nftfi.shape[0])
display(unique_to_april_nftfi.head(2))

In [ ]:
print(f"number of loans unique to MQ relative to NFTfi in April 2023: {unique_to_april_mq.shape[0]}, i.e. {100 * round(unique_to_april_mq.shape[0] / april_mq.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")

display(unique_to_april_mq.shape[0])
display(unique_to_april_mq.head(2))

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)
pd.DataFrame(unique_to_april_nftfi.groupby('loan_contract').agg(sum)['usd_value'])

In [ ]:
# MQ loans not in NFTfi
# unique_to_april_mq = april_mq[april_mq['loan_id_cleaned'].isin(mq_loans_not_in_nftfi)].reset_index(drop=True)
unique_to_april_mq.head(3)

In [ ]:
# We need to explore this table
unique_to_april_mq.erc20_name.unique()

## NFTfi loans not in MQ

In [ ]:
unique_to_april_nftfi

In [ ]:
# Explore the resulting set of unique_to_april_nftfi. Is there any pattern?
unique_to_april_nftfi.describe()

### Are discrepancies due to loan type which is not accounted for?

In [ ]:
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

In [ ]:
# Is the above true as well for the whole table set?
nftfi_loans_not_in_mq = list(
        set(list(nftfi.loan_no_cleaned)) - set(list(mq.loan_id_cleaned))
    )
print(len(nftfi_loans_not_in_mq))
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

### Is it due to missed erc20 token denomination?

In [ ]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    unique_to_april_nftfi.loc[unique_to_april_nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [ ]:
unique_to_april_nftfi.loan_erc20denomination_name.unique()
# It does not seem to be linked to missing token.

### Let's compute loan volumes per token denomination to see if they match

In [ ]:
display(april_nftfi.head(2))
display(april_mq.head(2))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', '{:,.2f}'.format)
display(april_nftfi.groupby('loan_erc20denomination_name').agg('sum'))

display(april_mq.groupby('erc20_name').agg('sum'))


In [ ]:

print("NFTfi:")
display(pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['loan_principal_amount']))
print("\n\nMQ:")
display(pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['principal_amount']))

#### We see from the above results that NFTfi and MQ respective loan principal amounts grouped by the loan erc20 denomination do not match at all. How about the total dollar numbers?

In [ ]:
print("NFTfi:")
total_usd_nftfi_volume_per_ccy = pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
total_usd_mq_volume_per_ccy = pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])
display(total_usd_mq_volume_per_ccy)

#### Total numbers are very off. Can that be explained with the set of loans which are not in each dataset?

In [ ]:
print("Total dollar value of loan principal for NFTfi loans which are not in MQ, for April and NFTfi:")
display(pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value']))
print("\n\nTotal dollar value of loan principal for MQ loans which are not inNFtfi, for April and MQ:")
display(pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd']))

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
usd_values_of_mq_not_in_nftfi = pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])

In [ ]:
weth_usd_amount_nftfi_not_in_mq = usd_values_of_nftfi_not_in_mq.loc['weth'].values[0]
weth_usd_amount_mq_not_in_nftfi = usd_values_of_mq_not_in_nftfi.loc['wETH'].values[0]

weth_usd_amount_total_nftfi = total_usd_nftfi_volume_per_ccy.loc['weth'].values[0]
weth_usd_amount_total_mq = total_usd_mq_volume_per_ccy.loc['wETH'].values[0]

print(weth_usd_amount_total_nftfi, weth_usd_amount_nftfi_not_in_mq, weth_usd_amount_mq_not_in_nftfi, weth_usd_amount_total_mq)

# total NFTfi missing from MQ volume, minus total MQ volume missing from NFTfi

weth_usd_amount_nftfi_not_in_mq - weth_usd_amount_mq_not_in_nftfi - weth_usd_amount_total_nftfi  + weth_usd_amount_total_mq

#### How about dollar amounts PER loan type PER ccy?

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby(['loan_erc20denomination_name','loan_type']).agg('sum')['usd_value'])
usd_values_of_nftfi_not_in_mq

In [ ]:
print("NFTfi:")
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
display(total_usd_mq_volume_per_ccy)

## For the loan IDs which do match... are they the same loans?

In [ ]:
import copy
nftfi_with_col_prefix = copy.deepcopy(nftfi)
mq_with_col_prefix = copy.deepcopy(mq)
nftfi_with_col_prefix = nftfi_with_col_prefix.rename(columns={col: 'nftfi_'+ col for col in nftfi.columns})
mq_with_col_prefix = mq_with_col_prefix.rename(columns={col: 'mq_'+ col for col in mq.columns})

matching_mq_with_nftfi = pd.merge(left=nftfi_with_col_prefix, right=mq_with_col_prefix, left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id_cleaned', how='inner', suffixes=('nftfi', 'mq'))
matching_mq_with_nftfi

In [ ]:
subset_of_matching_mq_with_nftfi = matching_mq_with_nftfi[['nftfi_date', 'mq_block_timestamp', 'nftfi_usd_value', 'mq_amt_in_usd', 'nftfi_loan_no_cleaned', 'nftfi_loan_type', 'mq_loan_id_cleaned']]
subset_of_matching_mq_with_nftfi

In [ ]:
pd.options.plotting.backend = "plotly"
subset_of_matching_mq_with_nftfi['dollar_diff'] = subset_of_matching_mq_with_nftfi['nftfi_usd_value'] - subset_of_matching_mq_with_nftfi['mq_amt_in_usd']

subset_of_matching_mq_with_nftfi.nftfi_date = pd.to_datetime(subset_of_matching_mq_with_nftfi.nftfi_date)
subset_of_matching_mq_with_nftfi.mq_block_timestamp = pd.to_datetime(subset_of_matching_mq_with_nftfi.mq_block_timestamp)

subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.dollar_diff)

In [ ]:
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.date_diff)

In [ ]:
import numpy as np
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
non_zero_time_delta = subset_of_matching_mq_with_nftfi.loc[subset_of_matching_mq_with_nftfi['date_diff'] > np.timedelta64(0)]
display(non_zero_time_delta)
display(non_zero_time_delta.nftfi_loan_type.unique())

print(f"min, max nftfi [{non_zero_time_delta.nftfi_date.min(), non_zero_time_delta.nftfi_date.max()}]")
print(f"min, max mq [{non_zero_time_delta.mq_block_timestamp.min(), non_zero_time_delta.mq_block_timestamp.max()}]")

print(f"\n\nTotal dollar diff for non zero time deltas (IDs not matching) [{non_zero_time_delta.dollar_diff.sum()}]")
print(f"Total dollar diff for both datasets [{subset_of_matching_mq_with_nftfi.dollar_diff.sum()}]")

In [ ]:
nftfi.date = pd.to_datetime(nftfi.date)
time_delta = pd.Timedelta(minutes=20)
datemax_minus_time_delta = non_zero_time_delta.nftfi_date.max() - time_delta
datemax_plus_time_delta = non_zero_time_delta.nftfi_date.max() + time_delta

print(non_zero_time_delta.nftfi_date.max())
nftfi.loc[nftfi.date < datemax_plus_time_delta].loc[datemax_minus_time_delta < nftfi.date].sort_values('date')

In [ ]:
non_zero_time_delta.loc[non_zero_time_delta['dollar_diff'] == 0]

### Are missing loans from MQ relative to NFTfi, bundles?

In [ ]:
display(unique_to_april_nftfi.collection_name.unique())
len(unique_to_april_nftfi.collection_name.unique())

In [ ]:
display(unique_to_april_mq.collection_address.unique()[:3])
len(unique_to_april_mq.collection_address.unique())

## How are liquidations accounted for?

In [ ]:
# TODO:
# spreadsheet of different transactions that are missing, or are more of them on MQ dataset. we need clear doc discussing which tsx are missing, which are added,
# what are the volume differences coming from.

# try and check that the eth value of those loans per transaction is the same.
# check if volumes per loanerc20 collateral are matching

## Sandbox

In [ ]:
try: 
    volume = pd.read_csv('analysis/nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
    base_data = pd.read_csv('analysis/nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
except FileNotFoundError:
    volume = (
        pd.read_csv('nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
        .drop(columns=['mean', 'median', 'std', 'count'])
    )
    base_data = (
        pd.read_csv('nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
        .drop(columns=['Unnamed: 0', 'Unnamed: 7', 'Unnamed: 8'])
    )

In [ ]:
display(volume.head(2))
display(base_data.head(2))

In [ ]:
nftfi = pd.merge(left=base_data, right=volume, on='collection_name')
nftfi 
# list(nftfi['borrow_volume'].values)